In [2]:
from sklearn.svm import SVR
from sklearn.model_selection import LeaveOneOut, GridSearchCV
import scipy.io
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [3]:
path_code = os.getcwd()
PA_dataset = scipy.io.loadmat(os.path.join(path_code, "../Data/PA_regr.mat"))

In [4]:
ar_m = PA_dataset["ar"]
af_m = PA_dataset["af"]
y_m = PA_dataset["slp"]


In [6]:
ar = np.zeros([len(ar_m[0]), 1])

In [16]:
ar_m[0][1]

array([[0.5422277]])

In [17]:
ar = np.zeros([len(ar_m[0]), 1])
for i in range(len(ar_m[0])):
    ar[i,0] = ar_m[0][i][0][0]

af = np.zeros([len(af_m[0]), 1])
for i in range(len(af_m[0])):
        af[i,0] = af_m[0][i][0][0]

y = np.zeros([len(y_m[0]), 1])
for i in range(len(y_m[0])):
        y[i,0] = y_m[0][i][0][0]

y = y.ravel()

X = np.stack((ar, af), axis=-1)

X = X.reshape(17, -1)


ValueError: cannot reshape array of size 38 into shape (17,newaxis)